In [1]:
from lol_data import get_summoner_lol_data
from tft_data import get_summoner_tft_data
from val_data import get_summoner_val_data
from config import API_KEY

game_name = input("Enter Game Name (lol, tft, valorant): ").lower()
summoner_name = input("Enter Summoner Name: ")
region = input("Enter Region (e.g., na1, euw1): ")

if game_name == 'lol':
    summoner_data = get_summoner_lol_data(API_KEY, summoner_name, region)
    if summoner_data:
        print(f"Summoner name: {summoner_data['name']}")
        print(f"Summoner Level: {summoner_data['summonerLevel']}")
        # Add more fields as needed
elif game_name == 'tft':
    summoner_data = get_summoner_tft_data(API_KEY, summoner_name, region)
    if summoner_data:
        print(f"TFT Summoner name: {summoner_data['name']}")
        # Add more fields as needed
elif game_name == 'valorant':
    summoner_data = get_summoner_val_data(API_KEY, summoner_name, region)
    if summoner_data:
        print(f"Valorant Summoner name: {summoner_data['name']}")
        # Add more fields as needed
else:
    print("Unknown game input. Please enter 'lol', 'tft', or 'valorant'.")


Error: 403, {"status":{"message":"Forbidden","status_code":403}}
